In [1]:
import numpy as np
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import cPickle as pickle
wikipedia_pickle = '../Data/WikiAstronauts/WikiAstronauts-DBpedia.p'
medline_pickle = '../Data/MedlinePlus/MedlinePlus-SemRep.p'

In [2]:
wiki_list = pickle.load( open( wikipedia_pickle, "rb" ) )
medline_list = pickle.load( open( medline_pickle, "rb" ) )

In [3]:
def clean_annotations(string):
    formal = True
    annotations = re.findall(r'(\[\[[0-9a-zA-Z_.,\-\(\)\%\'\|\ ]+\]\])', string)
    if len(annotations) > 0:
        for annotation in annotations:
            if formal:
                replacement = re.findall(r'\|([0-9a-zA-Z_.,\-\(\)\%\']+)\]\]', annotation)
            else:
                replacement = re.findall(r'\[\[([0-9a-zA-Z_.,\-\(\)\%\'\ ]+)\|', annotation)
            string = string.replace(annotation, replacement[0])
        clean_annotations(string)
    return string

In [4]:
namespaces = ['(http\:\/\/dbpedia\.org/\w+\/)', 'http\:\/\/www\.w3\.org\/1999\/02\/22\-rdf\-syntax\-ns\#', 'http\:\/\/www\.w3\.org\/2000\/01\/rdf\-schema\#']
def clean_namespace(triples):
    for j in range(0, len(triples)):
        for namespace in namespaces:
            triples[j] = re.sub(re.compile(namespace, re.DOTALL), '', triples[j])\
              .replace('\t', '').replace('\n', '') 
    return triples
print clean_namespace(wiki_list[0]['triples'])

['Soyuz_TM-6 crewLaunching Abdul_Ahad_Mohmand']


In [5]:
def highlight(triples):
    for j in range(0, len(triples)):
        triples[j] = triples[j].split()
        triples[j][1] = '<b><font color="red">' + triples[j][1] + '</font></b>'
        triples[j] = ' '.join(triples[j])
    return triples
print highlight((wiki_list[0]['triples']))[0]

Soyuz_TM-6 <b><font color="red">crewLaunching</font></b> Abdul_Ahad_Mohmand


In [6]:
def clean_annotations(original, triples):
    #print('Original')
    #print original, triples
    annotations = re.findall(r'(\[\[[0-9a-zA-Z_.,\-\(\)\%\'\|\ ]+\]\])', original)
    if len(annotations) > 0:
        for annotation in annotations:
            formal_part = re.findall(r'\|([0-9a-zA-Z_.,\-\(\)\%\']+)\]\]', annotation)[0]
            informal_part = re.findall(r'\[\[([0-9a-zA-Z_.,\-\(\)\%\'\ ]+)\|', annotation)[0]
            original = original.replace(annotation, informal_part)
            #print simplification
            #print formal_part
            found = False
            for i in range(0, len(triples)):
                triples[i] = triples[i].split()
                for j in range(0, len(triples[i])):
                    if formal_part == triples[i][j]:
                        triples[i][j] = informal_part
                triples[i] = ' '.join(triples[i])
                """
                if formal_part in triples[i]:
                    #namespace = re.findall(r'([0-9a-zA-Z_.,\-\(\)\%\'\:\/])+{}'.format(re.escape(formal_part)), triples[i])[0]
                    namespace = re.findall(r'({})'.format(re.escape(formal_part)), triples[i])[0]
                    triples[i] = triples[i].replace(namespace, informal_part)
                """
        clean_annotations(original, triples)
    return original, triples

In [7]:
def setup_experiment(list):
    num_sentences = 15
    incl_sentences = []
    while len(incl_sentences) < num_sentences:
        sentence = np.random.randint(0, len(list))
        init_triples = highlight(clean_namespace(list[sentence]['triples']))
        annotated_sentence, triples = clean_annotations(list[sentence]['annotated_sentence'].replace('\t', '').replace('\n', ''), \
                                              init_triples)
        triples = triples

        incl_sentences.append({'Original Sentence': annotated_sentence})
        incl_sentences[len(incl_sentences) - 1]['Triple-Fact 1'] = triples[0]         
        for j in range(1, len(list[sentence]['triples'])):
            incl_sentences[len(incl_sentences) - 1][('Triple-Fact %d' % (j + 1))] = triples[j]
        list.pop(sentence)
    return incl_sentences    

In [8]:
"""
def setup_experiment(list):
    num_sentences = 15
    incl_sentences = []
    while len(incl_sentences) < num_sentences:
        sentence = np.random.randint(0, len(list))
        init_triples = clean_namespace(list[sentence]['triples'])
        annotated_sentence, triples = clean_annotations(list[sentence]['annotated_sentence'].replace('\t', '').replace('\n', ''), \
                                              init_triples)

        incl_sentences.append({'Original Sentence': annotated_sentence, 'Triple-Facts': ', '.join(triples)})
        list.pop(sentence)
    return incl_sentences
"""

"\ndef setup_experiment(list):\n    num_sentences = 15\n    incl_sentences = []\n    while len(incl_sentences) < num_sentences:\n        sentence = np.random.randint(0, len(list))\n        init_triples = clean_namespace(list[sentence]['triples'])\n        annotated_sentence, triples = clean_annotations(list[sentence]['annotated_sentence'].replace('\t', '').replace('\n', ''),                                               init_triples)\n\n        incl_sentences.append({'Original Sentence': annotated_sentence, 'Triple-Facts': ', '.join(triples)})\n        list.pop(sentence)\n    return incl_sentences\n"

In [9]:
#output_df = pd.DataFrame(setup_experiment(wiki_list + medline_list))
output_df = pd.DataFrame(setup_experiment(wiki_list))
output_df.to_csv('../Data/Experiment.csv', index=False)